# 📊 Repositório de Consultas SQL para Analistas de Dados: **Departamento Financeiro**
-  🔍 Contribuições: Estamos abertos a contribuições da comunidade de analistas de dados! Se você tiver consultas SQL úteis que gostaria de compartilhar ou sugestões para melhorias, sinta-se à vontade para enviar um pull request. Juntos, podemos expandir e aprimorar este repositório para beneficiar toda a comunidade de análise de dados.
- 🎯 Sinta-se à vontade para explorar, utilizar e contribuir para este repositório, e que ele possa ser uma ferramenta valiosa em sua jornada como analista de dados!

# 📌 **Monitoramento de Gastos Financeiros com Classificação (Normal, Suspeito e Perigo)**

**Objetivo:**
O objetivo da query é calcular a média móvel de 30 dias dos gastos e classificar os gastos diários como "normal", "suspeito" ou "perigo", permitindo que o departamento financeiro monitore de forma eficaz as variações de gastos e identifique rapidamente anomalias que possam requerer ação imediata.

**Importância:**
- **Para o Departamento Financeiro:** Permite uma visão proativa e detalhada dos gastos, ajudando a identificar padrões e anomalias que possam indicar problemas ou oportunidades de otimização.
- **Para a Empresa:** A identificação precoce de gastos suspeitos ou perigosos pode ajudar a evitar fraudes, reduzir desperdícios e melhorar a saúde financeira da empresa.

### Query SQL

```sql
WITH DailyExpenses AS (
    SELECT
        expense_date,
        amount
    FROM
        financial_expenses
),
MovingAverage AS (
    SELECT
        expense_date,
        amount,
        AVG(amount) OVER (
            ORDER BY expense_date
            ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
        ) AS moving_avg_30d
    FROM
        DailyExpenses
),
ClassifiedExpenses AS (
    SELECT
        expense_date,
        amount,
        moving_avg_30d,
        CASE
            WHEN amount > moving_avg_30d * 1.3 THEN 'perigo'
            WHEN amount > moving_avg_30d THEN 'suspeito'
            ELSE 'normal'
        END AS classification
    FROM
        MovingAverage
)
SELECT
    expense_date,
    amount,
    moving_avg_30d,
    classification
FROM
    ClassifiedExpenses
ORDER BY
    expense_date;
```

### Explicação de Cada Etapa da Query

1. **DailyExpenses:**
    - **Objetivo:** Selecionar as colunas `expense_date` e `amount` da tabela `financial_expenses`.
    - **Importância:** Esta etapa cria uma base com os dados de despesas diárias.

2. **MovingAverage:**
    - **Objetivo:** Calcular a média móvel de 30 dias para cada data de despesa.
    - **Explicação do Código:**
        - `AVG(amount) OVER (ORDER BY expense_date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW)`: Calcula a média dos últimos 30 dias (incluindo o dia atual) para cada linha.
    - **Importância:** A média móvel ajuda a suavizar as flutuações diárias e fornece um valor de referência para identificar anomalias.

3. **ClassifiedExpenses:**
    - **Objetivo:** Classificar os gastos com base na média móvel calculada.
    - **Explicação do Código:**
        - `CASE WHEN amount > moving_avg_30d * 1.3 THEN 'perigo' WHEN amount > moving_avg_30d THEN 'suspeito' ELSE 'normal' END AS classification`: Classifica o gasto diário como 'perigo' (30% acima da média móvel), 'suspeito' (acima da média móvel) ou 'normal' (igual ou abaixo da média móvel).
    - **Importância:** A classificação facilita a identificação de despesas que requerem atenção especial.

4. **Seleção Final:**
    - **Objetivo:** Selecionar e ordenar os resultados finais.
    - **Explicação do Código:**
        - `SELECT expense_date, amount, moving_avg_30d, classification FROM ClassifiedExpenses ORDER BY expense_date`: Retorna as despesas diárias, a média móvel e a classificação, ordenadas por data.
    - **Importância:** Organiza os dados de maneira clara para análise ou para enviar alertas automáticos.

### Implementação do Bot

- **Alerta via E-mail e Slack:** Um bot pode ser configurado para verificar a tabela resultante e enviar alertas quando encontrar classificações 'suspeito' ou 'perigo'. Isso garante que a equipe financeira seja imediatamente informada de qualquer irregularidade, permitindo uma resposta rápida e eficaz.



# 📌 **EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization)**
- EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization) é um indicador financeiro que mede a lucratividade de uma empresa antes de deduções por juros, impostos, depreciação e amortização. Essencialmente, ele oferece uma visão clara da geração de caixa operacional da empresa, ignorando fatores que não estão diretamente ligados às operações diárias, como estrutura de capital e políticas fiscais. O EBITDA é amplamente utilizado para comparar a eficiência operacional entre empresas e indústrias, ignorando efeitos contábeis e fiscais.

```sql
SELECT
    DATE_TRUNC('month', data_pedido) AS mes,
    SUM(valor_total - custo_produtos - custo_operacional) AS ebitda
FROM pedidos
GROUP BY mes
ORDER BY mes;
```

# 📌 **Margem de Lucro Líquido**
- A Margem de Lucro Líquido é uma métrica financeira que representa a porcentagem do lucro líquido em relação à receita total de uma empresa. Ela é calculada subtraindo todos os custos, despesas operacionais, juros e impostos da receita total. Essa margem mostra quanto cada real de receita se converte em lucro após todos os gastos necessários para a operação da empresa serem cobertos. É um indicador crucial para avaliar a saúde financeira, a eficiência operacional e a capacidade de uma empresa gerar lucro a partir de suas receitas.

```sql
SELECT
    DATE_TRUNC('month', data_pedido) AS mes,
    (SUM(valor_total - custo_produtos - custo_operacional - impostos) / SUM(valor_total)) AS margem_lucro_liquido
FROM pedidos
GROUP BY mes
ORDER BY mes;
```

# 📌 **Margem de Lucro Bruto**
- A Margem de Lucro Bruto é uma métrica financeira que representa a porcentagem da receita total que resta após deduzir os custos dos bens vendidos (COGS - Cost of Goods Sold). Ela é calculada dividindo o lucro bruto pela receita total e é expressa em porcentagem. Essa margem é um indicador importante da eficiência operacional e rentabilidade de uma empresa, mostrando quanto dinheiro sobra das vendas após cobrir os custos diretos associados à produção dos bens ou serviços vendidos. Uma margem de lucro bruto alta sugere que a empresa é eficiente na gestão dos custos de produção e tem um bom controle sobre suas despesas operacionais.

```sql
SELECT
    DATE_TRUNC('month', data_pedido) AS mes,
    SUM(valor_total - custo_produtos) / SUM(valor_total) AS margem_lucro_bruto
FROM pedidos
GROUP BY mes
ORDER BY mes;
```

# 📌 **Gross Merchandise Volume (GMV)**
- O Gross Merchandise Volume (GMV) é a métrica que representa o total bruto das vendas de produtos ou serviços em uma plataforma durante um período específico. Ele calcula o volume total de transações comerciais realizadas, sem deduzir despesas como devoluções ou descontos. O GMV é frequentemente usado para medir a saúde financeira e o tamanho da atividade econômica em plataformas de comércio eletrônico e marketplaces, indicando o valor total dos bens que passaram pela plataforma.

```sql
SELECT
    DATE_TRUNC('month', data_pedido) AS mes,
    SUM(valor_total) AS gmv
FROM pedidos
GROUP BY mes
ORDER BY mes;
```

# 📌 **Taxa de Comissão Média**
- A Taxa de Comissão Média é uma métrica que calcula a média das taxas percentuais cobradas por uma plataforma ou serviço sobre as transações realizadas, como as vendas de restaurantes em um aplicativo de entrega. Ela é determinada dividindo a receita total obtida com as comissões pelo Gross Merchandise Volume (GMV), e é expressa em porcentagem. Essa taxa indica quão eficazmente a plataforma monetiza seus serviços ao cobrar dos usuários, como restaurantes, por exemplo, e é crucial para avaliar a rentabilidade e sustentabilidade financeira do modelo de negócios.

```sql
SELECT
    DATE_TRUNC('month', data_pedido) AS mes,
    AVG(taxa_comissao) AS taxa_comissao_media
FROM pedidos
GROUP BY mes
ORDER BY mes;
```

# 📌 **Churn Rate de Restaurantes**
- O Churn Rate de Restaurantes é uma métrica que mede a taxa de rotatividade ou o percentual de restaurantes parceiros que deixam de trabalhar com uma plataforma dentro de um período específico. Este indicador é crucial para entender a satisfação, a competitividade e a atratividade da plataforma para seus parceiros comerciais. Um churn rate alto pode sinalizar problemas como insatisfação com as políticas da plataforma, condições de serviço, ou a presença de concorrentes mais atraentes no mercado. Acompanhar essa taxa ajuda a plataforma a identificar áreas de melhoria para reter e atrair novos parceiros.

```sql
SELECT
    DATE_TRUNC('month', data_ultimo_pedido) AS mes,
    COUNT(DISTINCT restaurante_id) AS restaurantes_ativos,
    COUNT(DISTINCT CASE WHEN data_ultimo_pedido < CURRENT_DATE - INTERVAL '30 days' THEN restaurante_id END) AS restaurantes_churn
FROM pedidos
GROUP BY mes
ORDER BY mes;
```

# 📌 **Lifetime Value (LTV) do Cliente**
- O Lifetime Value (LTV) do Cliente é uma métrica financeira que estima o valor total de receita que uma empresa pode esperar de um único cliente ao longo de toda a sua relação com a empresa. O LTV ajuda a entender quanto um cliente vale em termos de receita e é crucial para tomar decisões sobre quanto investir em aquisição e retenção de clientes. Ele permite que as empresas avaliem a lucratividade a longo prazo de suas relações com os clientes e ajustem suas estratégias de marketing e vendas para otimizar esse valor.

```sql
SELECT
    cliente_id,
    AVG(valor_total) AS ltv
FROM pedidos
GROUP BY cliente_id;
```







# 📌 **Custo de Aquisição de Cliente (CAC)**
- O Custo de Aquisição de Cliente (CAC) é uma métrica financeira que calcula o custo total incorrido para adquirir um novo cliente. Inclui todos os gastos associados a marketing e vendas, divididos pelo número de clientes adquiridos durante um período específico. O CAC é fundamental para avaliar a eficácia e a eficiência das estratégias de marketing e vendas, ajudando as empresas a entender quanto estão investindo para ganhar cada novo cliente. Uma relação equilibrada entre o CAC e o Lifetime Value (LTV) dos clientes é crucial para garantir a sustentabilidade financeira da empresa.

```sql
SELECT
    DATE_TRUNC('month', data_cadastro) AS mes,
    SUM(custo_campanhas + custo_vendas) / COUNT(DISTINCT cliente_id) AS cac_mensal
FROM clientes
GROUP BY mes
ORDER BY mes;
```






